# Introduction to Conditional Probability 

One of the most important concepts in probability is *conditional probability*. This is the part of probability that is about determining how different events can affect each other. For data science/statistics, this is particularly important because we are often trying to determine whether an observed effect depends on some underlying feature or analyzing the nature of such dependence. 


Probability is generally considered a tricky subject, where answers defy initial intuitions, and there are many examples from conditional probability that fit this description. 

Let's start with a simple example that fools many who are new to probability or even those who have some basic knowledge of probability but who haven't studied conditional probability in detail:

## The Magician's Coin Part 1

Suppose you attend a magic show. The magician shows you two coins: a two-headed coin and a fair coin. She asks you to pick one of the coins and flip it at random, observing only the top face. If the outcome of that first flip is Heads, does that affect the probability that Heads would come up if you flip that coin again?

To be able to answer this question, we first need to know what is the probability of getting heads when one of the coins is chosen at random and flipped. I will show you how to easily analyze this probability in a minute, but let's first build a simple simulation to simulate this probability.

To facilitate learning, I build the simulation in three steps below. All of them leverage the `random` library, so let's start by importing that:

In [19]:
import random

**Step 1.** Let's use `random.choice()` to choose a random coin from the list `['fair', '2head']` and print it out for a very small simulation. (Note that there are **much** more computationally efficient ways to implement this simulation, but the goal here is to make the simulation as **clear as possible** to the reader.)

In [22]:
def choose_coin(num_sims=10):
    coins=['fair','2head']
    for sim in range(num_sims):
        coin=random.choice(coins)
        print(coin)

In [24]:
choose_coin()

2head
2head
2head
2head
fair
2head
2head
2head
fair
fair


**Step 2.** Now, we can check the outcome of the coin choice and randomly choose one of the faces of the chosen coin. If you are following along in a separate Jupyter notebook, I suggest you copy the function from above, rename it, and then add the rest of the code (the part below the comment in the function below):

In [14]:
def choose_and_flip(num_sims=10):
    coins=['fair','2head']
    for sim in range(num_sims):
        coin=random.choice(coins)
        # Delete the print statement and add the following:
        if coin=='fair':
            faces=['H','T'] 
        else:
            faces=['H','H']
        value=random.choice(faces)
        print(coin,value)

In [11]:
choose_and_flip()

2head H
fair T
fair T
fair H
fair T
2head H
2head H
2head H
fair T
fair T


By inspection, you should see that the relative frequency of heads is more than 1/2. That should be intuitive, and you may be able to guess the answer.

**Step 3.** Finally, let's estimate the probability of heads by determining the relative frequency of 'H'. We initialize a counter called `num_heads` to zero outside the simulation loop and then increment it every time the outcome is `H`. 

**Important: Note that we need to drastically increase the number of simulated coin flips, so we will turn off printing inside the for loop.**

In [15]:
def one_flip(num_sims=100_000):
    coins=['fair','2head']
    
    num_heads=0
    for sim in range(num_sims):
        coin=random.choice(coins)
        if coin=='fair':
            faces=['H','T'] 
        else:
            faces=['H','H']
        value=random.choice(faces)
        if value=='H':
            num_heads+=1
            
    print("Prob. of H is approximately", num_heads/num_sims)

In [17]:
one_flip()

Prob. of H is approximately 0.74921


How can we calculate this probability using equally likely outcomes? Let's create a table that enumerates all the outcomes:

              
 |Coin        | Face 1      | Face 2 |
 :-           | :-:         | :-: |
 Fair         | Heads       | Tails |
 Two-Headed   | Heads 1        | Heads 2 |
 
Here, I separately list the two different Heads outcomes for the two-headed coin because I want the outcomes to be equally likely. Since the coins are equally likely to be chosen, and the two faces of each coin are equally likely, the resulting outcomes of the combined experiment are also equally likely. 

Let $H_1$ denote the event that the top face of the coin on the first flip is Heads. From the table, $|H_1|=3$ and $|S|=4$. Since the outcomes are assumed to be equally likely, $P(H_1) = |H_1|/|S| = 3/4$, which matches our simulation.

The reader may wonder what happens if we instead ask about the second flip. Let $H_i$ be the event that the $i$th flip of the coin is Heads. If you do not already know what $P(H_2)$ will be, take a moment to think about what the simulation will look like -- with careful thought, you should be able to know $P(H_2)$. 

Let's build the simulation:

In [29]:
def two_flips(num_sims=100_000):
    coins=['fair','2head']
    
    num_heads=0
    for sim in range(num_sims):
        # Note that the coin is chosen once -- the same coin is used for both flips!!!
        coin=random.choice(coins)
        if coin=='fair':
            faces=['H','T'] 
        else:
            faces=['H','H']
        value1=random.choice(faces)
        value2=random.choice(faces)
        if value2=='H':
            num_heads+=1
            
    print("Prob. of H2 is approximately", num_heads/num_sims)

In [28]:
two_flips()

Prob. of H_2 is approximately 0.75101


Did you guess this answer? It should not be a surprise -- we now flip twice as many coins and then throw half of them away! So, the simulation is just a less efficient version of `one_flip()`.  If we don't have any other information about the coin (say, from observing the results of some of the flips), then the probability of heads on any flip is the same as $P(H_1)$: there are three possible heads outcomes out of four total outcomes.

**Comments on Things Not Observed**

Some students of probability have problems grasping the meaning of some of the probabilities asked about up to this point. For instance, students often ask questions like:
* How can we evaluate the probability of getting heads if we don't know which coin was chosen?
* How can we ask about the probability of getting heads on the $i$th flip, if we don't even know what the outcomes of flips $1$ through $i-1$ were?

It is normal to ask these questions, and we will be exploring this idea more as we learn about conditional probability. The short answer is that we can ask any question we like, but we may not always know how to answer it. A longer answer is that if we want to ask about the probability of some event $A$ that depends on some possible events $B_i$, we can find the probabilities of $A$ for each of the possible $B_i$ that could have occurred and then combine them appropriately to find the probability of $A$. We will define all this precisely and introduce the appropriate mathematics later in this section.

## The Magician's Coin Part 2

**JMS Working Here**

2. The magician withdraws a coin and flips it. If it comes up Heads, what is the probability that a second flip of that same coin will also be Heads?

How can we simulate this?

Do this before Thursday's class. Make a simulation two flips where you count:

* the number of time heads occurred on the first flip of the coin
* the number of times heads occured on a second flip of the coin when heads occurred on the first flip

We are looking for the ratio of the second counter to the first counter. I.e., we are ignoring all those cases where heads did not occur on the first flip because we are only asking about events where heads did occur on the first flip.

In [6]:
def two_flips(num_sims=10000):
    coins=['fair','2head']
    heads_count1=0
    heads_count2=0
    for sim in range(num_sims):
        coin=random.choice(coins)
        if coin=='fair':
            faces=['H','T']
        else:
            faces=['H','H']
        value=random.choice(faces)
        if value=='H':
            heads_count1+=1
            value=random.choice(faces)
            if value=='H':
                heads_count2+=1
    print("Prob. of heads on second flip given heads on first flip is",
          heads_count2/heads_count1)

In [5]:
two_flips()

Prob. of heads on second flip given heads on first flip is 0.8333111259160559


In [1]:
def one_flip2(num_sims=10000):
    coins=['fair','2head']
    heads_count=0
    two_head_count=0
    for sim in range(num_sims):
        coin=random.choice(coins)
        if coin=='fair':
            faces=['H','T']
        else:
            faces=['H','H']
        value=random.choice(faces)
        if value=='H':
            heads_count+=1
            if coin=='2head':
                two_head_count+=1
    print("Prob. of heads is",heads_count/num_sims)
    print("Prob of 2head coin when got heads is", two_head_count/heads_count)



In [5]:
one_flip2(100000)

Prob. of heads is 0.7502
Prob of 2head coin when got heads is 0.6700746467608638
